In [8]:
import pandas as pd
import matplotlib.pyplot as plt
from gensim import corpora
from gensim.models import Phrases
#https://www.datascience.com/resources/notebooks/word-embeddings-in-python
from collections import defaultdict
import nltk
from nltk.corpus import wordnet
import re
import numpy as np
from nltk.corpus import stopwords
import ast

In [34]:
df_snippets = pd.read_csv('datasets/00_snippets.csv')
df_snippets.head(2)

,claim_id,snippet_content,snippet_date,snippet_id,snippet_pagenum,snippet_title,snippet_url,claim_content,claim_date,claim_label,claim_tag,claim_url,date_number
0,3,"News on Japan, Business News, Opinion, Sports,...",17636,0,0,Article expired | The Japan Times,https://www.japantimes.co.jp/article-expired/,Black and Latino people in NYC are arrested at...,17646,True,— PolitiFact New York,/new-york/statements/2018/apr/25/kirsten-gilli...,0
1,3,Get the latest breaking news across the U.S. o...,17636,1,0,"U.S. News | Latest National News, Videos …",https://abcnews.go.com/US/,Black and Latino people in NYC are arrested at...,17646,True,— PolitiFact New York,/new-york/statements/2018/apr/25/kirsten-gilli...,0


In [39]:
s_quotes = ['`','´','ʹ','ʻ','ʼ','ʽ','ˊ','ˋ','˴','ʹ','΄','ՙ','՚','՛','՜','՝','‘','’','‛','′','‵', 'ߴ','י' ,'׳', 'ߵ']
d_quotes = ['ʺ','˝','ˮ','˵','˶','ײ','״', '“', '”', '‟', '″', '‴', '‶', '‷','``']

def replace_quotes(st):
    global s_quotes, d_quotes
    for c in s_quotes:
        st = st.replace(c,"'")
    for c in d_quotes:
        st = st.replace(c,'"')
    return st

df_snippets['snippet_content'] = df_snippets['snippet_content'].apply(lambda x: replace_quotes(x))
df_snippets['claim_content'] = df_snippets['claim_content'].apply(lambda x: replace_quotes(x))

In [40]:
garbage = "…[]•™💕🙂🇮🇩→♦☺~∼^★≈≥⌂℠„♫⊕†☆®©¬〇..."

def rm_garbage(st):
    global garbage
    for c in garbage:
        st = st.replace(c,'')
    return st

df_snippets['snippet_content'] = df_snippets['snippet_content'].apply(lambda x: rm_garbage(x))
df_snippets['claim_content'] = df_snippets['claim_content'].apply(lambda x: rm_garbage(x))

In [41]:
df_snippets['snippet_content'] = df_snippets['snippet_content'].apply(lambda x: x.lower())
df_snippets['claim_content'] = df_snippets['claim_content'].apply(lambda x: x.lower())

In [42]:
df_snippets['snippet_content'] = df_snippets['snippet_content'].apply(lambda x: x.replace('â€™s',''))
df_snippets['snippet_content'] = df_snippets['snippet_content'].apply(lambda x: x.replace('€€€',''))
df_snippets['snippet_content'] = df_snippets['snippet_content'].apply(lambda x: x.replace('â€œ',''))
df_snippets['snippet_content'] = df_snippets['snippet_content'].apply(lambda x: x.replace('â€',''))
df_snippets['snippet_content'] = df_snippets['snippet_content'].apply(lambda x: x.replace('â',''))
df_snippets['snippet_content'] = df_snippets['snippet_content'].apply(lambda x: x.replace('\\n',''))
df_snippets['snippet_content'] = df_snippets['snippet_content'].apply(lambda x: x.replace('\\r',''))
df_snippets['snippet_content'] = df_snippets['snippet_content'].apply(lambda x: x.replace('\\t',''))
df_snippets['snippet_content'] = df_snippets['snippet_content'].apply(lambda x: x.replace('\\',''))

df_snippets['claim_content'] = df_snippets['claim_content'].apply(lambda x: x.replace('â€™s',''))
df_snippets['claim_content'] = df_snippets['claim_content'].apply(lambda x: x.replace('€€€',''))
df_snippets['claim_content'] = df_snippets['claim_content'].apply(lambda x: x.replace('â€œ',''))
df_snippets['claim_content'] = df_snippets['claim_content'].apply(lambda x: x.replace('â€',''))
df_snippets['claim_content'] = df_snippets['claim_content'].apply(lambda x: x.replace('â',''))
df_snippets['claim_content'] = df_snippets['claim_content'].apply(lambda x: x.replace('\\n',''))
df_snippets['claim_content'] = df_snippets['claim_content'].apply(lambda x: x.replace('\\r',''))
df_snippets['claim_content'] = df_snippets['claim_content'].apply(lambda x: x.replace('\\t',''))
df_snippets['claim_content'] = df_snippets['claim_content'].apply(lambda x: x.replace('\\',''))

In [43]:
usa_abr = ['u.s.','u.s','u.s…','u.s.-','u.s.—','-u.s.', 
 'u.s..', '\'u.s']
unitednations_abr = ['u.n.']
losangeles_abr = ['l.a.', 'l.a','l.a.','l.a']
unitedkingdom_abr = ['u.k']
newyork_abr = ['n.y.','n.y']
europeanunion_abr = ['e.u']

def replace_abr(st):
    for abr in usa_abr:
        if abr in st:
            st = st.replace(abr,'united states')
    for abr in unitednations_abr:
        if abr in st:
            st = st.replace(abr,'united nations')
    for abr in losangeles_abr:
        if abr in st:
            st = st.replace(abr,'los angeles')
    for abr in unitedkingdom_abr:
        if abr in st:
            st = st.replace(abr, 'united kingdom')
    for abr in newyork_abr:
        if abr in st:
            st = st.replace(abr, 'new york')
    for abr in europeanunion_abr:
        if abr in st:
            st = st.replace(abr, 'european union')
    return st

df_snippets['snippet_content'] = df_snippets['snippet_content'].apply(lambda x: replace_abr(x))
df_snippets['claim_content'] = df_snippets['claim_content'].apply(lambda x: replace_abr(x))

In [44]:
str_sep = '*:•/_–—\\-‐,.|‑…→]―=[~+'

def rm_sep(st):
    for c in str_sep:
        if c in st:
            st = ' '.join(st.split(c))
    return st

df_snippets['snippet_content'] = df_snippets['snippet_content'].apply(lambda x: rm_sep(x))
df_snippets['claim_content'] = df_snippets['claim_content'].apply(lambda x: rm_sep(x))

In [45]:
df_snippets['snippet_content'] = df_snippets['snippet_content'].apply(lambda x: nltk.word_tokenize(x))
df_snippets['claim_content'] = df_snippets['claim_content'].apply(lambda x: nltk.word_tokenize(x))

In [46]:
df_snippets['snippet_content'] = df_snippets['snippet_content'].apply(lambda x: nltk.pos_tag(x))
df_snippets['claim_content'] = df_snippets['claim_content'].apply(lambda x: nltk.pos_tag(x))

In [47]:
print(list(set([x[1] for lst in df_snippets['claim_content'] for x in lst])))

['VBN', '.', 'VBZ', 'NNP', 'IN', 'VB', 'VBG', 'WDT', 'FW', 'RP', 'TO', '(', 'UH', "''", 'NNPS', 'NNS', 'NN', 'RB', 'DT', 'CC', 'WRB', 'EX', 'WP', '#', 'RBS', 'JJS', ':', 'PRP', ')', 'CD', 'JJR', 'MD', 'JJ', 'VBP', 'PDT', 'WP$', '``', 'POS', '$', 'VBD', 'RBR', 'PRP$']


In [48]:
worthless_pos_tags = [',', 'PRP$', 'WRB', 'RP', 'TO','(','SYM','WDT', '``', 'WP$', ')', 'EX',
                     'LS', ':', 'WP', 'MD', 'CD', '$', 'IN', '#', "''", 'FW', 'POS', 'DT',
                     '.', 'PDT', 'CC', 'UH', 'PRP']

df_snippets['snippet_content'] = df_snippets['snippet_content']\
.apply(lambda tb: list(filter(lambda x: x[1] not in worthless_pos_tags,tb)))

df_snippets['snippet_content'] = df_snippets['snippet_content'].apply(lambda tb: [x[0] for x in tb])

df_snippets['claim_content'] = df_snippets['claim_content']\
.apply(lambda tb: list(filter(lambda x: x[1] not in worthless_pos_tags,tb)))

df_snippets['claim_content'] = df_snippets['claim_content'].apply(lambda tb: [x[0] for x in tb])

In [49]:
pattern_interest = re.compile("^[a-z']+$")
df_snippets['snippet_content'] = df_snippets['snippet_content']\
.apply(lambda tb: list(filter(lambda x: pattern_interest.match(x), tb)))
df_snippets['claim_content'] = df_snippets['claim_content']\
.apply(lambda tb: list(filter(lambda x: pattern_interest.match(x), tb)))

In [50]:
bigrams = Phrases(list(df_snippets['snippet_content'].as_matrix()))

df_snippets['snippet_content'] = df_snippets['snippet_content'].apply(lambda tb: bigrams[tb])
df_snippets['claim_content'] = df_snippets['claim_content'].apply(lambda tb: bigrams[tb])

/home/ymerej/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [51]:
stopWords = set(stopwords.words('english'))

df_snippets['snippet_content'] = df_snippets['snippet_content']\
.apply(lambda tb: list(filter(lambda x: x not in stopWords, tb)))

df_snippets['claim_content'] = df_claims['claim_content']\
.apply(lambda tb: list(filter(lambda x: x not in stopWords, tb)))

In [18]:
df_snippets.to_csv('datasets/02_bow_snippets_claims.csv')

In [2]:
df_snippets = pd.read_csv('datasets/02_bow_snippets_claims.csv')

/home/ymerej/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
df_snippets['snippet_content'] = df_snippets['snippet_content'].apply(lambda x: ast.literal_eval(x))

In [13]:
dictionary = corpora.Dictionary(list(df_snippets['snippet_content'].as_matrix()))
print(len(dictionary.token2id))

194658


In [14]:
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n = 194658)
print(len(dictionary.token2id))

113764


In [22]:
for nb_word in sorted(list(np.arange(10000,40000,5000)),reverse=True):
    dictionary.filter_extremes(keep_n=nb_word)
    dictionary.save('dictionaries/02_'+str(nb_word/1000)+'.dict')
    print(len(dictionary.token2id))

35000
30000
25000
20000
15000
10000


In [24]:
dfclaims = df_snippets[['claim_id','claim_content','claim_label']].groupby(['claim_id'])\
.first().reset_index()[['claim_content','claim_label']]

In [30]:
nb_word = 35000
dictionary = corpora.Dictionary.load('dictionaries/02_'+str(nb_word/1000)+'.dict')
[len(dictionary.doc2bow(dfclaims['claim_content'][i])) for i in dfclaims.index[:10]]

[10, 7, 10, 12, 7, 11, 9, 7, 7, 13]

In [33]:
list(filter(lambda x: type(x) != type([]),list(dfclaims['claim_content'].as_matrix())))[:2]

[nan, nan]

In [25]:
for nb_word in sorted(list(np.arange(10000,40000,5000)),reverse=True):
    dictionary = corpora.Dictionary.load('dictionaries/02_'+str(nb_word/1000)+'.dict')
    dfclaims[str(nb_word)] = dfclaims['claim_content'].apply(lambda x: len(dictionary.doc2bow(x)))

TypeError: 'float' object is not iterable